<a href="https://colab.research.google.com/github/deepampatel/randomnlp/blob/main/IndianLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Oct  4 05:51:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz

--2020-10-04 05:51:51--  https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz
Resolving oscar-public.huma-num.fr (oscar-public.huma-num.fr)... 134.158.33.192
Connecting to oscar-public.huma-num.fr (oscar-public.huma-num.fr)|134.158.33.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 313843639 (299M) [application/x-gzip]
Saving to: ‘mr_dedup.txt.gz’

mr_dedup.txt.gz     100%[===================>] 299.30M  67.5MB/s    in 4.6s    

2020-10-04 05:51:56 (65.7 MB/s) - ‘mr_dedup.txt.gz’ saved [313843639/313843639]



In [3]:
!ls

drive  mr_dedup.txt.gz	sample_data


In [4]:
!gzip -d mr_dedup.txt.gz
!ls

drive  mr_dedup.txt  sample_data


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!wc -l mr_dedup.txt

1504068 mr_dedup.txt


In [7]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [8]:
!pip install tokenizers
!git clone https://github.com/huggingface/transformers
!pip install transformers/.


     |████████████████████████████████| 3.0MB 8.1MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 44859 (delta 2), reused 11 (delta 0), pack-reused 44842
Receiving objects: 100% (44859/44859), 32.19 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (31109/31109), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.0MB 7.8MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1082350 sha256=0d08ca4741faa702c39b016e90cdc3f6bb504597fad1cd29b30eab5a9216b44b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rwp0wrsn/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c

In [9]:
!head -15  mr_dedup.txt

Home / motivational marathi story / उद्योजकता (Entrepreneurship) / यांना हे जमलय, तर आपल्याला का नाही जमणार ?
यापैकी कोणाचीही सुरुवात स्वप्नवत झाली नव्हती. प्रत्येकाने संघर्ष केला आहे. आपल्या वेड्या स्वप्नांचा पाठलाग केला आहे. हे सर्वजण यशस्वी होऊ शकले कारण त्यांनी आपली स्वप्न पूर्ण करण्यासाठी "पाहिलं पाऊल" उचलण्याचं धाडस केलं !
आपल्या स्वप्नांचा पाठपुरावा केला तर यश मिळतेच. योग्य संधी शोधता यायला हवी किंवा आलेल्या संधीचा योग्य फायदा घेता आला पाहिजे. या सर्वांना ते जमलं , मग आपल्याला का नाही जमणार ?
वेटर्स बिचारे मोठ्यांच्या उपस्थितीमुळे चेहर्‍यावर कौतुकाचे भाव आणून लहान मुलांना मोठ्यांइतकाच आदर देताना दिसले. वेटर्सच काय, तर कॅप्टन्ससुद्धा!
मनात कुठेतरी दुखावलो गेलो. आईवडिलांनी चुकून माकून वर्षातून एकदा हॉटेलमध्ये नेले तर वडिलांशी दुर्मीळरीत्या कुजबुजत्या स्वरात आईलासुद्धा बोलताना पाहिलेला मी!
ही पिढी बिघडली ती आधीच्या पिढीमुळे, हे सार्वकालीन लागू होणारे विधान मान्य करूनसुद्धा हे जरा अतीच होत आहे असे वाटत राहिले.
हो, असं दिसतं खरं आजूबाजुला. आपल्या मुलांनी कसं वागावं हे त्यांच्या आईवडि

In [ ]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ["mr_dedup.txt"]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
#tokenizer.save_model(".", "marathibert")

In [ ]:
!mkdir martbertochar
tokenizer.save_model("martbertochar")

['martbertochar/vocab.json', 'martbertochar/merges.txt']

In [1]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./martbertochar/vocab.json",
    "./martbertochar/merges.txt",
)

In [2]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [3]:
op = tokenizer.encode("एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  ")

In [4]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()
import json

In [5]:
tokenizer.decode(op.ids)

'<s>एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  </s>'

In [23]:
!tail  +800000 mr_dedup.txt > output1.txt

In [24]:
!wc -l output1.txt

704069 output1.txt


In [6]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)

In [7]:
from transformers import RobertaTokenizerFast


tokenizer = RobertaTokenizerFast.from_pretrained("./martbertochar", max_len=512)

In [15]:
from transformers import RobertaForMaskedLM

#model = RobertaForMaskedLM(config=config)
model = RobertaForMaskedLM.from_pretrained('./martbertochar', return_dict=True)

In [16]:
model.num_parameters()
# => 84 million parameters

126031648

In [10]:
%%time
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="output1.txt",
    block_size=128,
)

CPU times: user 7min 9s, sys: 16.8 s, total: 7min 25s
Wall time: 2min 41s


In [11]:
from transformers import DataCollatorForLanguageModeling
??DataCollatorForLanguageModeling

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.20
)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./martbertochar",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_gpu_train_batch_size=48,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)


/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:267: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 1.1920186767578125, 'learning_rate': 4.957393142000137e-05, 'epoch': 0.034085486399890924, 'total_flos': 2323015335936000, 'step': 500}
{'loss': 1.1840626220703125, 'learning_rate': 4.914786284000273e-05, 'epoch': 0.06817097279978185, 'total_flos': 4646030671872000, 'step': 1000}
{'loss': 1.176662109375, 'learning_rate': 4.87217942600041e-05, 'epoch': 0.10225645919967279, 'total_flos': 6969046007808000, 'step': 1500}
{'loss': 1.172072509765625, 'learning_rate': 4.829572568000546e-05, 'epoch': 0.1363419455995637, 'total_flos': 9292061343744000, 'step': 2000}
{'loss': 1.165943359375, 'learning_rate': 4.786965710000682e-05, 'epoch': 0.17042743199945462, 'total_flos': 11615076679680000, 'step': 2500}
{'loss': 1.16050830078125, 'learning_rate': 4.744358852000818e-05, 'epoch': 0.20451291839934557, 'total_flos': 13938092015616000, 'step': 3000}
{'loss': 1.153578125, 'learning_rate': 4.7017519940009545e-05, 'epoch': 0.2385984047992365, 'total_flos': 16261107351552000, 'step': 3500}
{'

In [17]:
 trainer.save_model("./martbertochar")

In [19]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./martbertochar",
    tokenizer="./martbertochar"
)

In [ ]:
tokenizer.mask_token

'<mask>'

In [22]:
fill_mask("या गोष्टीचा <mask> ")

[{'score': 0.026546860113739967,
  'sequence': '<s>या गोष्टीचा एक </s>',
  'token': 338,
  'token_str': 'Ġà¤ıà¤ķ'},
 {'score': 0.009339679032564163,
  'sequence': '<s>या गोष्टीचाच </s>',
  'token': 277,
  'token_str': 'à¤ļ'},
 {'score': 0.007854795083403587,
  'sequence': '<s>या गोष्टीचा ( </s>',
  'token': 394,
  'token_str': 'Ġ('},
 {'score': 0.0071095433086156845,
  'sequence': '<s>या गोष्टीचा आवडत </s>',
  'token': 1128,
  'token_str': 'Ġà¤Ĩà¤µà¤¡à¤¤'},
 {'score': 0.0069724214263260365,
  'sequence': '<s>या गोष्टीचा ह </s>',
  'token': 299,
  'token_str': 'Ġà¤¹'}]

In [ ]:
`!mkdir /content/drive/My Drive/marathibert

mkdir: cannot create directory ‘/content/drive/My’: Operation not supported
mkdir: cannot create directory ‘Drive/marathibert’: No such file or directory


In [18]:
!cp -r /content/martbertochar/ /content/drive/My\ Drive/bigroberta

In [14]:
!cp -r /content/drive/My\ Drive/martbertochar/vocab*  /content/martbertochar/

In [ ]:
!rm -r /content/martbertochar/*.bin

In [12]:
!mkdir martbertochar